In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,root_mean_squared_error,mean_squared_error


In [5]:
df = pd.read_csv('dataframe')
inputs = ['FEED_BUT','FEED_PEN','FEED_HEX','DIST_RATE','Reflux Rate']
outputs = ['DIST_BUT','DIST_PEN','DIST_HEX',"Investment Cost",'Operating Cost']

X = df[inputs]
y = df[outputs]


mean_X = X.mean(axis=0)
std_X = X.std(axis=0)
print(std_X)
mean_y = y.mean(axis=0)
std_y = y.std(axis=0)
mean_y_weird = mean_y
std_y_weird = std_y
mean_X_weird = mean_X
std_X_weird = std_X

Xs = (X - mean_X) / std_X
ys = (y - mean_y) / std_y

X_train,X_test,y_train,y_test = train_test_split(Xs,ys, test_size=0.3, shuffle=True, random_state= 42)

X_train = np.asarray(X_train).astype('float64')
X_test = np.asarray(X_test).astype('float64')
y_train = np.asarray(y_train).astype('float64')
y_test = np.asarray(y_test).astype('float64')

FEED_BUT       155.738601
FEED_PEN       173.252653
FEED_HEX       173.378937
DIST_RATE      217.643283
Reflux Rate    160.163593
dtype: float64


In [6]:
#this data is simulated with n trays = 5 feed tray = 3 pressure = 1.9

nn_model = keras.Sequential([
      layers.Dense(5),
      #layers.Dense(32, activation='relu'),
      layers.Dense(32, activation='relu'),
      layers.Dense(5, activation = 'linear') 
])

nn_model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))

history = nn_model.fit(
    X_train,
    y_train,
    validation_split=0.1,
    verbose=0, epochs=350)

In [ ]:
results = nn_model.evaluate(X_test, y_test, verbose=0)
pred = nn_model.predict(X_test)

rmse = mean_squared_error(y_test,pred)
rmse = np.sqrt(rmse)

rmse_unconstr= mean_squared_error(y_test[:,3:],pred[:,3:])
rmse_unconstr = np.sqrt(rmse_unconstr)

rmse_constr= mean_squared_error(y_test[:,:3],pred[:,:3])
rmse_constr = np.sqrt(rmse_constr)

pred = pred*np.array(std_y) + np.array(mean_y)
test = y_test*np.array(std_y) + np.array(mean_y)

r2 = r2_score(test,pred)
r2_distbut = r2_score(test[:,0],pred[:,0])
r2_distpen = r2_score(test[:,1],pred[:,1])
r2_disthex = r2_score(test[:,2],pred[:,2])
r2_cap = r2_score(test[:,3],pred[:,3])
r2_oper = r2_score(test[:,4],pred[:,4])

#saving model to put into formulation. Either add path to Solving Formulation file or copy and paste .h5 file there.
nn_model.save('DIST simple model comp. 32 node.h5')

print("total rmse is {}".format(rmse))
print("unconstrained rmse is {}".format(rmse_unconstr))
print("constrained rmse is {}".format(rmse_constr))
print("total r2 is {}".format(r2))
print("r2 distillate butane is {}".format(r2_distbut))
print("r2 distillate pentane is {}".format(r2_distpen))
print("r2 distillate hexane is {}".format(r2_disthex))
print("r2 capital cost  is {}".format(r2_cap))
print("r2 operating cost  is {}".format(r2_oper))
